# Learning the difference between boys' and girls' names

## Text preprocessing

Goal: learn to tell the difference between different kinds of names (boys, girls, cities) using a 1-D CNN.

In [3]:
import numpy as np 

In [4]:
#Clean up the file by stripping out extra spaces
def cleanFile(inputFile, outputFile):

    namesFile = open(inputFile, "rw")
    fixedNamesFile = open(outputFile, "w")
    names = namesFile.readlines()   

    for i in range(len(names)):
        names[i] = names[i].strip()

    #print(names)
    for name in names:
        fixedNamesFile.write(name)    
        fixedNamesFile.write('\n')
    namesFile.close()
    fixedNamesFile.close()

#cleanFile("USGirlsNames.txt", "USGirlsNamesStripped.txt")

In [5]:
maxlen = 15 #The maximum name length that will be considered -- longer names will be truncated

In [6]:
def pad_name(n):
    return ("               "+n)[-maxlen:]


print( pad_name('fox') )

            fox


In [7]:
#now convert one string to an array of ascii codes

def convert_to_ascii(s):
   s=pad_name(s.upper().strip())
   return [ ord(c) for c in s ]
    

In [8]:
def convert_to_ascii_lists(names):
    return [convert_to_ascii(name) for name in names]



In [9]:
namesFile = open("USGirlsNamesStripped.txt", "rw")
names = namesFile.read().splitlines()   
asciiGirlNames = convert_to_ascii_lists(names)
namesFile.close()
#asciiGirlNames

In [10]:
namesFile = open("USBoysNamesStripped.txt", "rw")
names = namesFile.read().splitlines()
asciiBoyNames = convert_to_ascii_lists(names)
namesFile.close()

#print(asciiBoyNames)
print(type(asciiBoyNames))
print(asciiBoyNames[0])
print(type(asciiBoyNames[0]))


<type 'list'>
[32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 74, 65, 67, 79, 66]
<type 'list'>


In [11]:
import numpy as np

m=np.asarray( [ [1,2,3],[4,5]] )

print( m )
print( type(m) )
print( m[0])
print( type(m[0  ]))

[[1, 2, 3] [4, 5]]
<type 'numpy.ndarray'>
[1, 2, 3]
<type 'list'>


In [12]:
namesFile = open("Top5000PopulationStripped.csv", "rw")
names = namesFile.read().splitlines()
asciiCityNames = convert_to_ascii_lists(names)
namesFile.close()
#asciiCityNames
del names

## Building training and test sets

In [13]:
#To test all names vs city names, run this section: 
peopleY = np.ones(len(asciiBoyNames)+len(asciiGirlNames), dtype = np.int)
cityY = np.zeros(len(asciiCityNames), dtype = np.int)
ys = np.concatenate((peopleY, cityY), axis=0)
xs= np.concatenate((asciiBoyNames, asciiGirlNames, asciiCityNames), axis=0)

In [14]:
# Now we need to build training and test sets. We will test boys' names against girls' names first.
# Boys will be 0 and girls will be 1 :-) 
# Build the ys, join them to each other, and join the names to each other for the x.
# To separate them into training and test, create a permutation of integers 1:the length of y. 
# Then grab the top 200 (=10%) indexed by the permutation for the x and y test sets, and put the rest
# into the training sets.

#boysY = np.zeros(len(asciiBoyNames), dtype=np.int)
#girlsY = np.ones(len(asciiGirlNames), dtype=np.int)
#ys = np.concatenate((boysY, girlsY), axis=0)
#xs = np.concatenate((asciiBoyNames, asciiGirlNames), axis = 0)

#NOTE: the ys are *Not* integers. Will they need to be? Find out soon...

In [15]:
len(ys)

7000

In [16]:
testPercent = 0.1
i = np.random.permutation(len(ys))
testSize = int(testPercent*len(ys))

y_test = ys[i[0:testSize]]
y_train = ys[i[testSize:len(ys)]]
X_test = xs[i[0:testSize]]
X_train = xs[i[testSize:len(ys)]]
print(X_train)

#Yay! We have training and test sets!

[[32 32 32 ..., 73 79 78]
 [32 32 32 ..., 66 69 76]
 [32 32 32 ..., 84 79 78]
 ..., 
 [32 32 32 ..., 65 75 69]
 [32 32 32 ..., 75 69 69]
 [32 32 32 ..., 65 82 65]]


# Text classification

Now we begin the text classification using Convolution1D,
following the example of CNN_for_text_classification that ran on IMDB to classify sentiment
http://localhost:8888/notebooks/Documents/CNN_for_text_classification_example.ipynb

In [17]:
import numpy as np
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Convolution1D, GlobalMaxPooling1D

Using TensorFlow backend.


In [18]:
# set parameters:
max_features = 5000 #26 #Number of distinct letters?

batch_size = 32
embedding_dims = 50 #reduced from 50
nb_filter = 250 #reduced from 250. "Dimensionality of the output" -- but they are not making it binary so I don't know what this means...
filter_length = 3
hidden_dims = 250 #reduced from 250
nb_epoch = 3

#I don't know what nb_filter is.

In [19]:
#from keras.datasets import imdb
#(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
#maxlen=400


In [20]:
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
#print(X_train)

(6300, 'train sequences')
(700, 'test sequences')
Pad sequences (samples x time)
('X_train shape:', (6300, 15))
('X_test shape:', (700, 15))
('y_train shape:', (6300,))
('y_test shape:', (700,))


In [21]:
print(y_train[0:10])
print(type(X_train))
print(type(y_test))
print(type(X_test[0]))
print(X_train)

[0 1 0 0 1 0 0 0 0 0]
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
<type 'numpy.ndarray'>
[[32 32 32 ..., 73 79 78]
 [32 32 32 ..., 66 69 76]
 [32 32 32 ..., 84 79 78]
 ..., 
 [32 32 32 ..., 65 75 69]
 [32 32 32 ..., 75 69 69]
 [32 32 32 ..., 65 82 65]]


In [22]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen,
                    dropout=0.2))

# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(nb_filter=nb_filter,
                        filter_length=filter_length,
                        border_mode='valid',
                        activation='relu',
                        subsample_length=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))



Build model...


TypeError: where() takes at most 2 arguments (3 given)

In [44]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [45]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
validation_data=(X_test, y_test))

Train on 6300 samples, validate on 700 samples
Epoch 1/3
6300/6300 [==============================] - 2s - loss: 0.4698 - acc: 0.7711 - val_loss: 0.3606 - val_acc: 0.8557

(Accuracy: ~82% for all first names vs city names; 70% for male vs female names)